In [1]:
import sympy as sym
import sympy.physics.mechanics as me
from sympy.abc import t as time
import pandas as pd
import seaborn as sns

import numpy as np
from scipy.integrate import solve_ivp
import matplotlib.pyplot as plt

import sys, os

sys.path.insert(1, os.path.join(sys.path[0], '../..'))
import custom_plot_objects as cpo

import ModelFramework as mf
import ModelFramework.Elements as ele
import ModelFramework.ExternalForces as ef
import FwtModels.RectWing as rw
import FwtModels.TwoDoF as TwoDof
import FwtModels as fm

from matplotlib.lines import Line2D

me.mechanics_printing()

In [2]:
half = sym.Rational(1,2)

p = TwoDof.base_params()

In [44]:
# p = mf.ModelParameters.DynamicModel(2)
# p.y_0 = sym.Symbol('y_0')

# p.m_w = mf.ModelSymbol(value = 1,string = 'm_b')      # mass of ball
# p.m = mf.ModelSymbol(value = 1,string = 'm_p')      # mass of pendulum
# p.l_com = mf.ModelSymbol(value = 1,string = 'l_p')      # pivot
# p.k = mf.ModelSymbol(value = 0,string = 'k')      # spring const
# p.g  = mf.ModelSymbol(value = 9.81,string = 'g')                     # gravity

#define refernce frames
wing_root_frame = mf.HomogenousTransform().R_x(-p.q[1])
wing_aero_frame = wing_root_frame.Translate(p.y_0,0,0)
wing_com_frame = wing_root_frame.Translate(0,p.l_com,0)

#Create Elemnts
M_fwt = ele.MassMatrix(p.m,I_xx = 0)
M_innerWing = ele.RigidElement.PointMass(wing_root_frame,p.m_w,True)

fwt_ele = ele.RigidElement(wing_com_frame,M_fwt,True)
#spring_ele = ele.Spring(p.q[0],p.k_w)
#spring_ele_fwt = ele.Spring(p.q[1],p.k_fwt)


# Create AeroForces
wing_AeroForces = ef.AeroForce.PerUnitSpan(p,wing_aero_frame,p.a_0,
                               alphadot = p.alphadot_1,
                               M_thetadot = p.M_thetadot,
                               e = 0,
                               w_g = p.w_g,
                               rootAlpha = p.alpha_1,
                               alpha_zero = 0,
                               c = p.c_root-((p.c_root-p.c_tip)*p.y_0/p.s),
                            c_d_max = p.c_d_max)#.integrate((p.y_0,0,p.s))

# Setup AoA of FWT
fwt_aoa = fm.GetAoA(p.alpha_r,0,p.Lambda,p.q[1])

## Sub in Aero Forces
wing_AeroForces = wing_AeroForces.subs({p.alpha_1:fwt_aoa,p.alphadot_1:fwt_aoa.diff(time)})

forces = [wing_AeroForces]
forces = []
segments = 5
for i in range(segments):
    seg_width = p.s/segments
    yi = seg_width/2 + i*seg_width
    forces.append(wing_AeroForces.subs({p.y_0:yi})*seg_width)
Q = sym.Matrix([0]*p.qs)
for f in forces:
    Q += f.Q()
c_forces = ef.ExternalForce(Q)

In [45]:
#sm = mf.SymbolicModel.FromElementsAndForces(p,[M_innerWing,fwt_ele,spring_ele,spring_ele_fwt],c_forces)
sm = mf.SymbolicModel.FromElementsAndForces(p,[M_innerWing,fwt_ele],c_forces)

In [46]:
sm.M

⎡0     0    ⎤
⎢           ⎥
⎢        2  ⎥
⎣0  l_com ⋅m⎦

In [9]:
sm.to_file('2Dof_Model.py')